In [ ]:
import time
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from marker_analyser.io import load_oscillations_from_directory

from lumicks import pylake

import numpy as np

In [ ]:
dir_base = Path("/Volumes/shared/newton_lab/Data/Metallodrugs/CuOda/andrew")
assert dir_base.exists()

# Load oscillations
exclude_curve_ids = [
    "CuOda_1uM_M4_1",
    "CuOda_10uM_M4_1",
    "CuOda_0uM_M4_1",
    "CuOda_0uM_M4_2",
    "CuOda_0uM_M4_3",
    "CuOda_10uM_M5_1",
    "CuOda_0uM_M5_3",
    "CuOda_10uM_M6_1",
    "CuOda_0uM_M6_3",
    "CuOda_10uM_M7_1",
    "CuOda_0uM_M7_3",
    "CuOda_20uM_M8_1",
    "CuOda_0uM_M8_3",
    "CuOda_20uM_M9_1",
    "CuOda_0uM_M9_3",
    "CuOda_20uM_M10_1",
    "CuOda_0uM_M10_3",
    "CuOda_40uM_M11_1",
    "CuOda_0uM_M11_1",
    "CuOda_0uM_M11_2",
    "CuOda_40uM_M12_1",
    "CuOda_0uM_M12_1",
    "CuOda_0uM_M12_2",
    "CuOda_40uM_M13_1",
    "CuOda_0uM_M13_1",
    "CuOda_0uM_M13_2",
    "CuOda_40uM_M14_1",
    "CuOda_0uM_M14_1",
    "CuOda_0uM_M14_2",
]
metadata_regex = (
    r"^(?P<drug>[A-Za-z][\w\-]*)_(?P<conc>\d+(?:\.\d+)?(?:pM|nM|uM|μM|mM))_[Mm](?P<marker>\d+)_(?P<repeat>\d+)$"
)
oscillations = load_oscillations_from_directory(
    dir_base,
    exclude_curve_ids=exclude_curve_ids,
    metadata_regex=metadata_regex,
    force_maximum=20.0,
    distance_minimum=0.0,
)
oscillations_unified_model = oscillations.model_copy(deep=True)
oscillatinons_individual_model = oscillations.model_copy(deep=True)

In [ ]:
# Fit models to all oscillations
# oscillations.fit_model_to_all(segment="increasing")

# fit one model to all oscillations, but separated by the curve id
# so I think that the repeats in each fd curve will share the same parameters
# and fit together?
# also this fits both the increasing and decreasing segments together?
# marina's code doesn't split it up like mine does.
start_time = time.perf_counter()
model = pylake.ewlc_odijk_force("DNA") + pylake.force_offset("DNA")
fit_object = pylake.FdFit(model)
for oscillation_id, oscillation in oscillations_unified_model.items():

    fit_object.add_data(
        name=oscillation_id,
        f=np.concatenate([oscillation.increasing_force, oscillation.decreasing_force]),
        d=np.concatenate([oscillation.increasing_distance, oscillation.decreasing_distance]),
        params={
            "DNA/Lp": f"DNA/Lp_{oscillation.curve_id}",
            "DNA/Lc": f"DNA/Lc_{oscillation.curve_id}",
            "DNA/St": f"DNA/St_{oscillation.curve_id}",
            "DNA/f_offset": f"DNA/f_offset_{oscillation.curve_id}",
        },
    )

print(f"fitting the model")
fit_object.fit()

# For each oscillation, fit the model to the distance data
fig, ax = plt.subplots(figsize=(10, 10))
for oscillation_id, oscillation in oscillations_unified_model.items():
    distances = np.concatenate([oscillation.increasing_distance, oscillation.decreasing_distance])
    this_sample_params = {
        "DNA/Lp": fit_object.params[f"DNA/Lp_{oscillation.curve_id}"].value,
        "DNA/Lc": fit_object.params[f"DNA/Lc_{oscillation.curve_id}"].value,
        "DNA/St": fit_object.params[f"DNA/St_{oscillation.curve_id}"].value,
        "DNA/f_offset": fit_object.params[f"DNA/f_offset_{oscillation.curve_id}"].value,
        "kT": fit_object.params["kT"].value,
    }

    modelled_forces = model(independent=distances, params=this_sample_params)
    oscillation.increasing_fitted_forces = modelled_forces[: len(oscillation.increasing_distance)]
    oscillation.decreasing_fitted_forces = modelled_forces[len(oscillation.increasing_distance) :]
    ax.scatter(
        distances,
        np.concatenate([oscillation.increasing_force, oscillation.decreasing_force]),
        s=5,
        label=f"{oscillation_id} data",
    )
    ax.plot(distances, modelled_forces, label=oscillation_id, linewidth=2)
ax.set_title("After Fitting one model with n parameter sets")
ax.set_ylabel("Force [pN]")
ax.set_xlabel(r"Distance [$\mu$m]")
# ax.legend(loc="upper left", bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()
end_time = time.perf_counter()
print(f"Time taken to fit unified model: {end_time - start_time:.2f} seconds")

In [ ]:
# fit one model per oscillation
start_time = time.perf_counter()
for oscillation_id, oscillation in oscillatinons_individual_model.items():
    model = pylake.ewlc_odijk_force("DNA") + pylake.force_offset("DNA")
    fit_object = pylake.FdFit(model)
    fit_object.add_data(
        name=oscillation_id,
        f=np.concatenate([oscillation.increasing_force, oscillation.decreasing_force]),
        d=np.concatenate([oscillation.increasing_distance, oscillation.decreasing_distance]),
    )
    fit_object.fit()
    # fit the model to the data
    distances = np.concatenate([oscillation.increasing_distance, oscillation.decreasing_distance])
    modelled_forces = model(independent=distances, params=fit_object.params)
    oscillation.increasing_fitted_forces = modelled_forces[: len(oscillation.increasing_distance)]
    oscillation.decreasing_fitted_forces = modelled_forces[len(oscillation.increasing_distance) :]

# plot the modelled fits and data for all oscillations like before
fig, ax = plt.subplots(figsize=(10, 10))
for oscillation_id, oscillation in oscillatinons_individual_model.items():
    distances = np.concatenate([oscillation.increasing_distance, oscillation.decreasing_distance])
    ax.scatter(
        distances,
        np.concatenate([oscillation.increasing_force, oscillation.decreasing_force]),
        s=5,
        label=f"{oscillation_id} data",
    )
    ax.plot(
        distances,
        np.concatenate([oscillation.increasing_fitted_forces, oscillation.decreasing_fitted_forces]),
        label=oscillation_id,
        linewidth=2,
    )
ax.set_title("After Fitting Individually")
ax.set_ylabel("Force [pN]")
ax.set_xlabel(r"Distance [$\mu$m]")
# ax.legend(loc="upper left", bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()
end_time = time.perf_counter()
print(f"Time taken to fit individual models: {end_time - start_time:.2f} seconds")

In [ ]:
# Check if the fits are the same between the two methods
# gridspec of plots, two square on the top row, one long rectangle on the bottom row
fig = plt.figure(constrained_layout=True, figsize=(12, 8))
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
max_fitted_force = -np.inf
for oscillation_id in oscillations.keys():
    oscillation_individual = oscillatinons_individual_model[oscillation_id]
    oscillation_unified = oscillations_unified_model[oscillation_id]
    distances = np.concatenate([oscillation_unified.increasing_distance, oscillation_unified.decreasing_distance])
    fitted_forces_unified = np.concatenate(
        [oscillation_unified.increasing_fitted_forces, oscillation_unified.decreasing_fitted_forces]
    )
    fitted_forces_individual = np.concatenate(
        [oscillation_individual.increasing_fitted_forces, oscillation_individual.decreasing_fitted_forces]
    )

    max_fitted_force = max(max_fitted_force, np.max(fitted_forces_unified), np.max(fitted_forces_individual))

    modelled_forces_diffs = fitted_forces_unified - fitted_forces_individual
    ax1.plot(distances, fitted_forces_unified)
    ax2.plot(distances, fitted_forces_individual)
    ax3.plot(distances, modelled_forces_diffs)
    ax4.plot(distances, modelled_forces_diffs)
ax4.set_ylim(-1, max_fitted_force)
ax1.set_title("Unified Fit (one fit model with n parameter sets, one for each curve)")
ax1.set_xlabel("Distance [um]")
ax1.set_ylabel("Force [pN]")
ax2.set_title("Individual Fit - one fit model per curve")
ax2.set_xlabel("Distance [um]")
ax2.set_ylabel("Force [pN]")
ax3.set_title("Difference between Unified and Individual Fits (zoomed)")
ax3.set_xlabel("Distance [um]")
ax3.set_ylabel("Force Difference [pN]")
ax4.set_title("Difference between Unified and Individual Fits (in height context)")
ax4.set_xlabel("Distance [um]")
ax4.set_ylabel("Force Difference [pN]")
plt.show()